<a href="https://colab.research.google.com/github/ereinha/Deep-KAN/blob/main/RBFKAN2D_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from RBF_KAN import *
from RBF_KAN_2D import *

In [ ]:
# Load MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
valset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=False)

In [ ]:
# Define model
rbf2d = RBFKAN2D(layers_hidden_x=[28, 28], layers_hidden_y=[28, 28], num_grids=(8,8), use_base_update=True)
rbf1d = RBFKAN(layers_hidden=[28*28, 64, 10])
model = nn.Sequential(rbf2d, nn.Flatten(start_dim=1), rbf1d)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=5e-4)

# Define loss
criterion = nn.CrossEntropyLoss()

# Define ReduceLROnPlateau scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

for epoch in range(20):
    # Train
    model.train()
    total_loss = 0
    total_accuracy = 0
    with tqdm(trainloader) as pbar:
        for images, labels in pbar:
            images = images.to(device).squeeze(1)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            accuracy = (output.argmax(dim=1) == labels).float().mean()
            total_loss += loss.item()
            total_accuracy += accuracy.item()
            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item())
    total_loss /= len(trainloader)
    total_accuracy /= len(trainloader)

    # Validation
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for images, labels in valloader:
            images = images.to(device).squeeze(1)
            labels = labels.to(device)
            output = model(images)
            val_loss += criterion(output, labels).item()
            val_accuracy += (output.argmax(dim=1) == labels).float().mean().item()
    val_loss /= len(valloader)
    val_accuracy /= len(valloader)

    # Step the scheduler based on validation loss
    scheduler.step(val_loss)

    print(f"Epoch {epoch + 1}, Train Loss: {total_loss}, Train Accuracy: {total_accuracy}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
100%|██████████| 938/938 [00:24<00:00, 38.22it/s, accuracy=0.906, loss=0.295]


Epoch 1, Train Loss: 0.31573774219989015, Train Accuracy: 0.9052838486140725, Val Loss: 0.14635395767778206, Val Accuracy: 0.9575039808917197


100%|██████████| 938/938 [00:21<00:00, 42.92it/s, accuracy=0.969, loss=0.0955]


Epoch 2, Train Loss: 0.12667743687523897, Train Accuracy: 0.9618203624733476, Val Loss: 0.10395291419388358, Val Accuracy: 0.9683519108280255


100%|██████████| 938/938 [00:21<00:00, 42.90it/s, accuracy=1, loss=0.0654]


...

Epoch 18, Train Loss: 0.0021535107219594965, Train Accuracy: 0.9994502931769723, Val Loss: 0.07173594157738243, Val Accuracy: 0.9823845541401274


100%|██████████| 938/938 [00:21<00:00, 43.38it/s, accuracy=1, loss=0.00242]


Epoch 19, Train Loss: 0.0015804446298868587, Train Accuracy: 0.9997001599147122, Val Loss: 0.09910225663912037, Val Accuracy: 0.9748208598726115


100%|██████████| 938/938 [00:19<00:00, 47.12it/s, accuracy=1, loss=0.00152]


Epoch 20, Train Loss: 0.008759030132467796, Train Accuracy: 0.9972847814498934, Val Loss: 0.0791881067022549, Val Accuracy: 0.9794984076433121


In [ ]:
# Define model
model = RBFKAN(layers_hidden=[28 * 28, 64, 10])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

# Define loss
criterion = nn.CrossEntropyLoss()

# Define ReduceLROnPlateau scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

for epoch in range(20):
    # Train
    model.train()
    total_loss = 0
    total_accuracy = 0
    with tqdm(trainloader) as pbar:
        for images, labels in pbar:
            images = images.view(-1, 28 * 28).to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            accuracy = (output.argmax(dim=1) == labels).float().mean()
            total_loss += loss.item()
            total_accuracy += accuracy.item()
            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item())
    total_loss /= len(trainloader)
    total_accuracy /= len(trainloader)

    # Validation
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for images, labels in valloader:
            images = images.view(-1, 28 * 28).to(device)
            labels = labels.to(device)
            output = model(images)
            val_loss += criterion(output, labels).item()
            val_accuracy += (output.argmax(dim=1) == labels).float().mean().item()
    val_loss /= len(valloader)
    val_accuracy /= len(valloader)

    # Step the scheduler based on validation loss
    scheduler.step(val_loss)

    print(f"Epoch {epoch + 1}, Train Loss: {total_loss}, Train Accuracy: {total_accuracy}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

100%|██████████| 938/938 [00:18<00:00, 50.14it/s, accuracy=0.969, loss=0.143]


Epoch 1, Train Loss: 0.46666163372110203, Train Accuracy: 0.8611407249466951, Val Loss: 0.27851956134816264, Val Accuracy: 0.917296974522293


100%|██████████| 938/938 [00:19<00:00, 48.56it/s, accuracy=0.906, loss=0.238]


Epoch 2, Train Loss: 0.28419494114593785, Train Accuracy: 0.9171441897654584, Val Loss: 0.2759908712142782, Val Accuracy: 0.9152070063694268


100%|██████████| 938/938 [00:18<00:00, 50.07it/s, accuracy=0.906, loss=0.516]


...

Epoch 18, Train Loss: 0.07008761147618976, Train Accuracy: 0.9777785181236673, Val Loss: 0.15414431179090624, Val Accuracy: 0.9583001592356688


100%|██████████| 938/938 [00:18<00:00, 50.59it/s, accuracy=1, loss=0.0283]


Epoch 19, Train Loss: 0.06468018197091117, Train Accuracy: 0.9798607409381663, Val Loss: 0.1497129723522367, Val Accuracy: 0.9567078025477707


100%|██████████| 938/938 [00:18<00:00, 49.58it/s, accuracy=1, loss=0.0189]


Epoch 20, Train Loss: 0.06406086827315557, Train Accuracy: 0.9797441364605544, Val Loss: 0.15598580241203308, Val Accuracy: 0.9561106687898089
